In [1]:
import pandas as pd
from labels import LABELS

In [2]:
data = "/teamspace/studios/this_studio/max/dataset_resume.csv"
df = pd.read_csv(data, usecols=LABELS + ['description', 'resume'])


In [3]:
def setup(df_orig) -> pd.DataFrame:
    df = df_orig.copy()

    df["label"] = pd.Series().fillna("")
    for index, row in df.iterrows():
        # s'il y a un résumé, on l'utilise comme description
        df.loc[index, "description"] = (
                row.resume
                if isinstance(row.resume, str)
                else row.description
            )

        features = row[LABELS]
        df.at[index, "label"] = [feature for feature in features]

    df = df.rename(columns={'description': 'text'})
    return df[['text', 'label']]

features = df[LABELS]

df_clean = setup(df)

In [4]:
import torch
from transformers import CamembertTokenizer, AutoTokenizer
from transformers import CamembertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df_clean['text'], df_clean['label'], test_size=.2, random_state=42)

In [7]:
train_texts,train_labels

(291    Le Parti pris des choses / Exposition collecti...
 63     Dimanche 17 Septembre de 8h à 18h, organisée p...
 239    DÉLIRES DE NOËL dimanche 17 décembre 2023 de 1...
 353    📣 Le Trail des Pipi-Malo 2023 sera de retour l...
 219    Spectacle de théâtre "Extinction" avec Julien ...
                              ...                        
 71     La Chèvrerie La Bêle Fontenoise organise la 3è...
 106    Salon Destination Bien Être de Caudry les 21 e...
 270    SALON DES ARTISTES AMATEURS AULNESIENS Du 29 m...
 348    « Tous en forêt » Un rendez-vous privilégié av...
 102    Ce salon qui, d'année en année, monte en puiss...
 Name: text, Length: 312, dtype: object,
 291    [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 63     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 239    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
 353    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
 219    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
                              .